### Purchase Prediction Problem - Collaborative Filtering

After some difficulty getting matrix factorization to work, I am now attempting to follow a tutorial I found online [here](https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6).

To further help, I have cut the training data in half by removing users who do not appear in the test data.

In [1]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.cross_validation import train_test_split


ModuleNotFoundError: No module named 'turicreate'